# Preprocesamiento del Dataset: Agregar Columnas de Estación y Parte del Día

Este Jupyter Notebook tiene como objetivo realizar el preprocesamiento de un dataset al agregar dos nuevas columnas: "estación" y "parte_del_día" basadas en la columna "date_time".

El dataset de entrada es un archivo CSV con la siguiente estructura:
- Path_Final: Ruta del archivo de imagen
- Station: Identificador de la estación
- cam: Identificador de la cámara
- date_time: Fecha y hora de la imagen
- class: Clase de la imagen

Los pasos involucrados en el preprocesamiento del dataset son los siguientes:

1. Leer el archivo CSV de entrada utilizando la función `pd.read_csv()`.
2. Extraer el mes y la hora de la columna "date_time" utilizando la función `pd.to_datetime()` con el formato especificado.
3. Agregar una nueva columna "estación" basada en los valores del mes, de acuerdo con las siguientes condiciones:
   - Invierno: diciembre a febrero (meses 12, 1, 2)
   - Primavera: marzo a mayo (meses 3, 4, 5)
   - Verano: junio a agosto (meses 6, 7, 8)
   - Otoño: septiembre a noviembre (meses 9, 10, 11)
4. Agregar una nueva columna "parte_del_día" basada en los valores de la hora, de acuerdo con las siguientes condiciones:
   - Mañana: 6 a 11
   - Tarde: 12 a 20
   - Noche: 21 a 5
5. Renombrar la columna "Path_Final" a "path" para mayor consistencia.
6. Crear un nuevo DataFrame con las columnas seleccionadas: "path", "Station", "cam", "date_time", "estación", "parte_del_día" y "class".
7. Guardar el dataset preprocesado como un nuevo archivo CSV.


In [1]:
# Librerias
import pandas as pd

In [2]:
# Constantes

# Ruta de los archivos CSV
csv_path = "../../CSVs/c02_images_data_class.csv"

### Leer el CSVs

In [3]:
# Leer el archivo CSV
df = pd.read_csv(csv_path)

### Extraer revisión

In [4]:
# Extraer la columna 'rev' de la columna 'Path_Final'
df['rev'] = df['Path_Final'].str.split('/').str[0]

### Extraer hora y mes

In [5]:
# Extraer el mes y la hora de la columna 'date_time'
df['month'] = pd.to_datetime(df['date_time'], format='%Y:%m:%d %H:%M:%S').dt.month
df['hour'] = pd.to_datetime(df['date_time'], format='%Y:%m:%d %H:%M:%S').dt.hour

### Establecer estación

In [6]:
# Definir las condiciones para la columna 'season'
conditions_season = [
    (df['month'].isin([12, 1, 2])),
    (df['month'].isin([3, 4, 5])),
    (df['month'].isin([6, 7, 8])),
    (df['month'].isin([9, 10, 11]))
]
choices_season = ['winter', 'spring', 'summer', 'autumn']
df['season'] = pd.np.select(conditions_season, choices_season, default='unknown')


C:\Users\albam\AppData\Local\Temp\ipykernel_25984\4040015362.py:9: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df['season'] = pd.np.select(conditions_season, choices_season, default='unknown')


### Establecer parte del día

In [7]:
# Definir las condiciones para la columna 'day_part'
conditions_day_part = [
    (df['hour'].between(6, 11)),
    (df['hour'].between(12, 20)),
    (~df['hour'].between(6, 20))
]
choices_day_part = ['morning', 'evening', 'night']
df['day_part'] = pd.np.select(conditions_day_part, choices_day_part, default='unknown')

C:\Users\albam\AppData\Local\Temp\ipykernel_25984\1543899450.py:8: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df['day_part'] = pd.np.select(conditions_day_part, choices_day_part, default='unknown')


In [8]:
# Renombrar la columna 'Path_Final' a 'path'
df = df.rename(columns={'Path_Final': 'path'})
# Renombrar la columna 'Station' a 'station'
df = df.rename(columns={'Station': 'station'})

In [9]:
# Eliminar las columnas innecesarias
df.drop(['month', 'hour'], axis=1, inplace=True)

In [10]:
# Reordenar
df = df[['path', 'class', 'rev', 'station', 'cam', 'season', 'day_part', 'date_time']].copy()

### Guardar el DataFrame en un nuevo archivo CSV

In [11]:
# Definir la ruta y el nombre del archivo de salida
output_file = "../../CSVs/d01_images_data_amplified.csv"

# Guardar el DataFrame filtrado en un nuevo archivo CSV
df.to_csv(output_file, index=False)
